In [1]:
import torch
import os
import numpy as np
from torch.utils.data import Dataset, DataLoader
import random
import librosa
import soundfile as sf
from tqdm import tqdm

torch.cuda.is_available()
torch.cuda.device_count()
torch.cuda.current_device()
torch.cuda.device(0)
torch.cuda.get_device_name(0)

# check if files can be opened
print("PyTorch Version: ",torch.__version__)
# Detect if we have a GPU available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
if torch.cuda.is_available():
    print("Using the GPU!")
else:
    print("WARNING: Could not find GPU! Using CPU only.")

PyTorch Version:  2.1.0
Using the GPU!


In [2]:
class CustomDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.file_list = []
        
        self.labels_to_int = {}
        labels = sorted(os.listdir(root_dir))
        for i in range(len(labels)):
            self.labels_to_int[labels[i]] = i

        # Iterate through each class folder
        for class_folder in os.listdir(root_dir):
            class_path = os.path.join(root_dir, class_folder)
            
            # Check if it's a directory
            if os.path.isdir(class_path):
                # Iterate through CSV files in the class folder
                for file_name in os.listdir(class_path):
                    if file_name.endswith('.csv'):
                        file_path = os.path.join(class_path, file_name)
                        self.file_list.append((file_path, class_folder))

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, idx):
        file_path, label = self.file_list[idx]
        data = np.genfromtxt(file_path, delimiter=',')
        
        #random crop
        if(data.shape[1] > 100):
            rng = random.randrange(0, data.shape[1]-100)
            data = torch.tensor(data[:, rng:rng+100], dtype=torch.float32)
        else:
            data = torch.tensor(data, dtype=torch.float32)
        
        label = self.labels_to_int[label]
        sample = {'data': data, 'label': label}

        if self.transform:
            sample = self.transform(sample)

        return sample


In [3]:
root_dir = 'C:\GenshinVoice\Melspec_data'
custom_dataset = CustomDataset(root_dir)
dataloader = DataLoader(custom_dataset, batch_size=64, shuffle=False)

In [4]:
# to train here
for i, batch in tqdm(enumerate(dataloader)):
    data, labels = batch['data'], batch['label']
    
    # put train here, just a place holder now
    aa = data[2].numpy()
    aaa = librosa.feature.inverse.mel_to_audio(aa, sr = 22050)
    out_file_name = 'C:\\Users\\Joshua Ning\\Desktop\\VChanger\\test' + str(i) + '.wav'
    sf.write(out_file_name, aaa, 22050)
    i = i+1
    if i == 3:
        break

2it [00:04,  2.20s/it]
